In [ ]:
import whisperx
import gc
import os
import json

In [ ]:
device = 'cpu'
batch_size = 8
compute_type = 'int8'

model = whisperx.load_model("base", device, compute_type=compute_type, language='en')

In [ ]:
rootpath = '/Users/lindseytepfer/Dartmouth College Dropbox/Lindsey Tepfer/ctom/data/audio/v1/'
pairlist = [x for x in os.listdir(rootpath) if 'pair' in x]

In [ ]:
for pairid in pairlist[0:1]:
    audiofiles = os.listdir(rootpath+pairid)

    print("starting with:", pairid)

    for wav in audiofiles[0:1]:
        if len(audiofiles) > 0:
            fname = wav.split('.')[0]
            print('transcribing:', wav)

            fpath = rootpath+pairid+os.sep+wav
            batch_size = 16 # reduce if low on GPU mem
        
            print(fpath)

            #Unaligned transcriptions
            audio = whisperx.load_audio(fpath)
            result = model.transcribe(audio, batch_size=batch_size)
        
            #Aligned transcriptions
            model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
            result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)
            
            with open(fname+".json","w") as write_file:
                json.dump(result["segments"],write_file)
